In [28]:
from sklearn.neighbors import kneighbors_graph
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import os

In [29]:
class Graph():
 
    def __init__(self, vertices):
        self.V = vertices
        self.graph = [[0 for column in range(vertices)]
                      for row in range(vertices)]
 
    def printSolution(self, dist):
        print("Vertex tDistance from Source")
        for node in range(self.V):
            print(node, "t", dist[node])
 
    # A utility function to find the vertex with
    # minimum distance value, from the set of vertices
    # not yet included in shortest path tree
    def minDistance(self, dist, sptSet):
 
        # Initilaize minimum distance for next node
        min = sys.maxsize
 
        # Search not nearest vertex not in the
        # shortest path tree
        for v in range(self.V):
            if dist[v] < min and sptSet[v] == False:
                min = dist[v]
                min_index = v
 
        return min_index
 
    # Funtion that implements Dijkstra's single source
    # shortest path algorithm for a graph represented
    # using adjacency matrix representation
    def dijkstra(self, src):
 
        dist = [sys.maxsize] * self.V
        dist[src] = 0
        sptSet = [False] * self.V
 
        for cout in range(self.V):
 
            # Pick the minimum distance vertex from
            # the set of vertices not yet processed.
            # u is always equal to src in first iteration
            u = self.minDistance(dist, sptSet)
 
            # Put the minimum distance vertex in the
            # shotest path tree
            sptSet[u] = True
 
            # Update dist value of the adjacent vertices
            # of the picked vertex only if the current
            # distance is greater than new distance and
            # the vertex in not in the shotest path tree
            for v in range(self.V):
                if (self.graph[u][v] > 0) and (sptSet[v] == False) and (dist[v] > dist[u] + self.graph[u][v]):
                    dist[v] = dist[u] + self.graph[u][v]
 
        return dist

def get_topk(a, k):
    # return the indices of top k largest values in array a
    #return a.argsort()[-k:]
    return np.argpartition(a, -k)[-k:]

def get_kneighbor_graph(d, k):
    #return a adj matrix with 0 and 1 representing a graph
    #d: distant matrix
    # edge(u,v) = 1 if the distance from u to v is in the top k largest distances from u
    # the returned matrix is symmetric
    V = len(d)
    a = np.zeros((V,V), dtype=int)
    for u in range(V):
        kneighbor = get_topk(d[u], k)
        for v in kneighbor:
            a[u][v] = 1
    for i in range(V):
        for j in range(i +1, V):
            if a[i][j] != 0:
                a[j][i] = a[i][j]
            if a[j][i] != 0:
                a[i][j] = a[j][i]
    return a

def get_adj_matrix_of_graph(d, k0):
#return an adj matrix of a connected graph
    k = k0
    n_nodes = len(d)
    g = Graph(n_nodes)
    while True:    
        A = get_kneighbor_graph(d,k)
        B = A.copy()
        B[B == 0] = n_nodes
        g.graph = B
        
        #checking connectivity of the graph
        found = False
        
        for i in range(n_nodes):
            dist = g.dijkstra(i)
            
            if np.max(dist) >= n_nodes:
                found = True
                break # the graph is not connected
                
        if not found: # the graph is connected
            return k, A
        else:
            k += 1
        if k== n_nodes:
            return k, None

        
def create_METISgraph_file(d_file, cl_data_size_fname):
    z_dir = "./z_ass/"

    print("creating a graph file for METIS...")

    if not os.path.exists(z_dir):
        os.makedirs(z_dir)
    
    d = pd.read_csv(z_dir + d_file, index_col=False, header=None).to_numpy()
    cl_data_size = pd.read_csv(z_dir + cl_data_size_fname, header=None).to_numpy()
    
    n_nodes = len(d)
    print ("n_nodes, n_clients: ", n_nodes, ", ", len(cl_data_size))
    
    k, A = get_adj_matrix_of_graph(d, 5)
    
    
    if k >= n_nodes:
        print("Cannot create a connected graph!")
        exit()
    
    num_edges = np.sum(A) / 2
    
    header = str(n_nodes) + " " + str(int(num_edges)) + " 011\n"

    fname = "g_no_vw_knn" + str(k) + "_" + d_file

    with open(z_dir + fname, 'w') as the_file:
        the_file.write(header)

    for i in range (n_nodes):

        i_prime = i + 1
        #a_line = str(cl_data_size[i][0])
        a_line = str(1)
        for j in range (n_nodes):
            j_prime = j + 1
            if j_prime == i_prime:
                continue
            if A[i][j] == 1:
                a_line = a_line + " " + str(j_prime) + " " + str(int(A[i][j]))

        a_line = a_line + "\n"
        with open(z_dir + fname, 'a') as the_file:
            the_file.write(a_line)

    the_file.close()
    print("saved graph in file: " + z_dir + fname)
    return k, A

In [30]:
#main
d_fname = "d_euclidean_CNN_P1.6m_B10_L20_non_iid_zipfz0.7"
cl_data_size_fname = "client_data_size_CNN_P1.6m_B10_non_iidzipfz0.7"
k, A = create_METISgraph_file(d_fname, cl_data_size_fname)


creating a graph file for METIS...
n_nodes, n_clients:  50 ,  50
saved graph in file: ./z_ass/g_no_vw_knn5_d_euclidean_CNN_P1.6m_B10_L20_non_iid_zipfz0.7


In [8]:
k

1

In [102]:
d = np.array([[0, 3, 4, 5], [3, 0, 5, 4], [4, 5, 0, 3], [5, 4, 3, 0]])
k, A = get_adj_matrix_of_graph(d, 1)
print(k)
print(A)

k  1
B  [[4 4 4 1]
 [4 4 1 4]
 [4 1 4 4]
 [1 4 4 4]]
i, dist  0 [0, 4, 4, 1]
k  2
B  [[4 4 1 1]
 [4 4 1 1]
 [1 1 4 4]
 [1 1 4 4]]
i, dist  0 [0, 2, 1, 1]
i, dist  1 [2, 0, 1, 1]
i, dist  2 [1, 1, 0, 2]
i, dist  3 [1, 1, 2, 0]
2
[[0 0 1 1]
 [0 0 1 1]
 [1 1 0 0]
 [1 1 0 0]]


In [104]:
a = np.asarray([[0, 0, 1, 1],
 [0, 0, 1, 1],
 [1, 1, 0, 0],
 [1, 1, 0, 0]])
np.sum(a)

8

In [61]:
X = [[0, 4, 3, 5], [4, 0, 5, 3], [3, 5, 0, 4], [5, 3, 4, 0]]
X = np.array([[0, 1], [1.01, 1.], [2, 0], [3, 6], [9, 8], [10, 8], [9,20]])

neigh = NearestNeighbors(n_neighbors=2)
neigh.fit(X)
#NearestNeighbors(n_neighbors=2)

A = neigh.kneighbors_graph(X, 2, mode='distance')
A.toarray()

array([[ 0.        ,  1.01      ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 1.01      ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.40716026,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  5.38145891,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , 12.        ,
         0.        ,  0.        ]])

In [62]:
A = kneighbors_graph(X, 2, mode='distance', include_self=False)
A.toarray()

array([[ 0.        ,  1.01      ,  2.23606798,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 1.01      ,  0.        ,  1.40716026,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 2.23606798,  1.40716026,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 5.83095189,  5.38145891,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  6.32455532,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  7.28010989,  1.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , 12.        ,
        12.04159458,  0.        ]])

In [19]:
# Python program for Dijkstra's single
# source shortest path algorithm. The program is
# for adjacency matrix representation of the graph
 
# Library for INT_MAX
import sys
 
class Graph():
 
    def __init__(self, vertices):
        self.V = vertices
        self.graph = [[0 for column in range(vertices)]
                      for row in range(vertices)]
 
    def printSolution(self, dist):
        print("Vertex tDistance from Source")
        for node in range(self.V):
            print(node, "t", dist[node])
 
    # A utility function to find the vertex with
    # minimum distance value, from the set of vertices
    # not yet included in shortest path tree
    def minDistance(self, dist, sptSet):
 
        # Initilaize minimum distance for next node
        min = sys.maxsize
 
        # Search not nearest vertex not in the
        # shortest path tree
        for v in range(self.V):
            if dist[v] < min and sptSet[v] == False:
                min = dist[v]
                min_index = v
 
        return min_index
 
    # Funtion that implements Dijkstra's single source
    # shortest path algorithm for a graph represented
    # using adjacency matrix representation
    def dijkstra(self, src):
 
        dist = [sys.maxsize] * self.V
        dist[src] = 0
        sptSet = [False] * self.V
 
        for cout in range(self.V):
 
            # Pick the minimum distance vertex from
            # the set of vertices not yet processed.
            # u is always equal to src in first iteration
            u = self.minDistance(dist, sptSet)
 
            # Put the minimum distance vertex in the
            # shotest path tree
            sptSet[u] = True
 
            # Update dist value of the adjacent vertices
            # of the picked vertex only if the current
            # distance is greater than new distance and
            # the vertex in not in the shotest path tree
            for v in range(self.V):
                if (self.graph[u][v] > 0) and (sptSet[v] == False) and (dist[v] > dist[u] + self.graph[u][v]):
                    dist[v] = dist[u] + self.graph[u][v]
 
        return dist
        #self.printSolution(dist)
 
 
# Driver program
g = Graph(4)
g.graph = [[0, 1, 5, 5],[1,0,5,1],[5,5,0,5],[5,1,5,0]]
g.dijkstra(0)
 

Vertex tDistance from Source
0 t 0
1 t 1
2 t 5
3 t 2


In [47]:
d = pd.read_csv("./z_ass/d_euclidean_MLP2_B10_L200_non_iid_zipfz0.7")
cl_data_size = pd.read_csv("./z_ass/client_data_size_MLP2_B10_non_iidzipfz0.7")

In [48]:
reverse_d = d.to_numpy()
reverse_d = np.max(reverse_d) - reverse_d
reverse_d

array([[99.        , 99.        , 53.25149689, ..., 55.49250104,
        55.63961572, 55.51368946],
       [98.        , 53.25149689, 99.        , ..., 83.53824091,
        83.47237176, 83.54756917],
       [97.        , 55.63301446, 83.87808133, ..., 94.47277716,
        93.77537274, 94.84202577],
       ...,
       [ 2.        , 55.49250104, 83.53824091, ..., 99.        ,
        94.33274411, 95.29361813],
       [ 1.        , 55.63961572, 83.47237176, ..., 94.33274411,
        99.        , 94.44774011],
       [ 0.        , 55.51368946, 83.54756917, ..., 95.29361813,
        94.44774011, 99.        ]])

In [49]:
A = kneighbors_graph(reverse_d, 5, mode='connectivity', include_self=False)
A.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [45]:
#A = A.astype(int)
#A[A == 0] = 1000
#print (A)

  (0, 0)	1000
  (0, 1)	1000
  (0, 2)	1000
  (0, 3)	1
  (0, 4)	1000
  (0, 5)	1000
  (0, 6)	1
  (0, 7)	1000
  (0, 8)	1000
  (0, 9)	1000
  (0, 10)	1000
  (0, 11)	1000
  (0, 12)	1
  (0, 13)	1000
  (0, 14)	1000
  (0, 15)	1000
  (0, 16)	1000
  (0, 17)	1000
  (0, 18)	1000
  (0, 19)	1
  (0, 20)	1000
  (0, 21)	1000
  (0, 22)	1000
  (0, 23)	1000
  (0, 24)	1000
  :	:
  (99, 75)	1000
  (99, 76)	1000
  (99, 77)	1000
  (99, 78)	1000
  (99, 79)	1000
  (99, 80)	1000
  (99, 81)	1000
  (99, 82)	1000
  (99, 83)	1000
  (99, 84)	1000
  (99, 85)	1000
  (99, 86)	1000
  (99, 87)	1000
  (99, 88)	1000
  (99, 89)	1000
  (99, 90)	1000
  (99, 91)	1000
  (99, 92)	1
  (99, 93)	1000
  (99, 94)	1
  (99, 95)	1
  (99, 96)	1
  (99, 97)	1
  (99, 98)	1000
  (99, 99)	1000


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: SparseEfficiencyWarning: Comparing a sparse matrix with 0 using == is inefficient, try using != instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [40]:
print (np.sum(A))

500.0


In [50]:
A = A.astype(int)
print(A[0][0:10])

  (0, 3)	1
  (0, 6)	1
  (0, 12)	1
  (0, 19)	1
  (0, 40)	1


In [52]:
print(A[3][0:10])

  (0, 0)	1
  (0, 19)	1
  (0, 28)	1
  (0, 40)	1
  (0, 57)	1


In [42]:
g = Graph(100)
g.graph = A
g.dijkstra(0)


<100x100 sparse matrix of type '<class 'numpy.float64'>'
	with 500 stored elements in Compressed Sparse Row format>

In [ ]:
def get_adj_matrix_of_graph(d)
    k = 3
    n_nodes = 100
    g = Graph(n_nodes)
    while True:    
        A = kneighbors_graph(d, k, mode='connectivity', include_self=False)
        B = A.toarray().astype(int)
        B[B == 0] = n_nodes
        g.graph = B
        #checking connectivity of the graph
        for i in range(n_nodes):
            dist = g.dijktra(i)
            if np.max(dist) >= n_nodes:
                break # the graph is not connected
        if i == n_nodes: # the graph is connected
            return k, A
        else:
            k += 1
        if k== n_nodes:
            return None

In [ ]:
def create_METISgraph_file(d):
        z_dir = ./z_ass/
        
        print("creating a graph file for METIS...")
        
        if not os.path.exists(self.z_dir):
            os.makedirs(self.z_dir)
        
        n_nodes = 100 
        num_edges = n_nodes * k / 2
        header = str(len(self.clients)) + " " + str(int(num_edges)) + " 011\n"
        
        if self.zipfz != None:
            d_fname = "d_euclidean_" + self.model_name + "_B" +str(self.batch_size) + "_L" + str(no_local_epochs) + "_"+ self.mode +"_zipfz" + str(self.zipfz)
        else:    
            d_fname = "d_euclidean_" + self.model_name + "_B" + str(self.batch_size) + "_L" + str(no_local_epochs)
        
        fname = "g_" + d_fname

        with open(self.z_dir + fname, 'w') as the_file:
            the_file.write(header)
        
        d = (1000 * self.client_distance_matrix()).astype(int)
        
        for i in range (len(self.clients)):
            
            i_prime = i + 1
            a_line = str(len(self.clients[i].data))
            
            for j in range (len(self.clients)):
                j_prime = j + 1
                if j_prime == i_prime:
                    continue
                a_line = a_line + " " + str(j_prime) + " " + str(int(d[i][j]))
            
            a_line = a_line + "\n"
            with open(self.z_dir + fname, 'a') as the_file:
                the_file.write(a_line)
        
        the_file.close()
        print("saved graph in file: " + self.z_dir + fname)